In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=888a4dadf9f5600b842e074e0a2c8a0d82a4fd248aed134ac1effb7a89edee15
  Stored in directory: /home/ng/.cache/pip/wheels/b1/91/5f/283b53010a8016a4ff1c4a1edd99bbe73afacb099645b5471b
Successfully built pyspark


In [56]:

from pyspark.sql import SparkSession

from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [2]:
# Tạo một SparkSession
spark = SparkSession.builder \
    .appName("VDT DE 2024") \
    .getOrCreate()

24/06/12 02:39:09 WARN Utils: Your hostname, ng-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.117.128 instead (on interface ens33)
24/06/12 02:39:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 02:39:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#write file danh_sach_sv
ds = spark.read.csv("./data/danh_sach_sv_de.csv", header=True, inferSchema=True)
ds.coalesce(1).write.mode('overwrite').option('header','true').csv('hdfs://localhost:9010/raw_zone/fact/ds_de_vdt/danh_sach_sv_de.csv')

In [6]:
studentActivity = StructType() \
                    	.add("student_code", "integer")\
                    	.add("activity", "string")\
                    	.add("numberOfFile", "integer")\
                    	.add("timestamp", "string")



In [22]:


# Đường dẫn tới file Parquet trong HDFS
parquet_file_path = "hdfs://localhost:9010/raw_zone/fact/activity/"

# Đọc file Parquet từ HDFS
df = spark.read.schema(schema=studentActivity).parquet(parquet_file_path)


df.show()



+------------+--------+------------+---------+
|student_code|activity|numberOfFile|timestamp|
+------------+--------+------------+---------+
|          17| execute|          10|6/13/2024|
|          11| execute|           9|6/13/2024|
|          31| execute|           2|6/15/2024|
|           3| execute|           9|6/13/2024|
|          28| execute|           8|6/14/2024|
|           5| execute|           3|6/10/2024|
|          24| execute|           8|6/12/2024|
|          28| execute|           4|6/14/2024|
|          16| execute|           9|6/13/2024|
|          33| execute|           1|6/10/2024|
|           8| execute|           2|6/14/2024|
|          13| execute|          10|6/14/2024|
|          37| execute|           9|6/12/2024|
|          38| execute|           9|6/11/2024|
|          30| execute|           2|6/15/2024|
|          39| execute|           1|6/13/2024|
|          32| execute|           5|6/10/2024|
|          19| execute|           5|6/14/2024|
|           2

In [23]:
df1 = df

In [28]:
df1.count()

400

In [60]:
df= df.withColumn("timestamp", date_format(to_date(df["timestamp"], "M/d/yyyy"), "yyyyMMdd"))
df.show()

+------------+--------+------------+---------+
|student_code|activity|numberOfFile|timestamp|
+------------+--------+------------+---------+
|          17| execute|          10| 20240613|
|          11| execute|           9| 20240613|
|          31| execute|           2| 20240615|
|           3| execute|           9| 20240613|
|          28| execute|           8| 20240614|
|           5| execute|           3| 20240610|
|          24| execute|           8| 20240612|
|          28| execute|           4| 20240614|
|          16| execute|           9| 20240613|
|          33| execute|           1| 20240610|
|           8| execute|           2| 20240614|
|          13| execute|          10| 20240614|
|          37| execute|           9| 20240612|
|          38| execute|           9| 20240611|
|          30| execute|           2| 20240615|
|          39| execute|           1| 20240613|
|          32| execute|           5| 20240610|
|          19| execute|           5| 20240614|
|           2

In [62]:
df_procesed = df.groupBy("student_code", "activity", "timestamp").agg(sum("numberOfFile").alias("total_files"))
df_procesed.show()

+------------+--------+---------+-----------+
|student_code|activity|timestamp|total_files|
+------------+--------+---------+-----------+
|          16| execute| 20240613|          9|
|          11| execute| 20240613|         17|
|          24| execute| 20240612|         18|
|          12| execute| 20240611|         16|
|          17| execute| 20240613|         13|
|           8| execute| 20240614|          6|
|          31| execute| 20240615|          9|
|          39| execute| 20240613|          6|
|          13| execute| 20240614|         26|
|          37| execute| 20240612|          9|
|          17| execute| 20240610|         14|
|          32| execute| 20240610|          5|
|          19| execute| 20240612|          4|
|           3| execute| 20240613|          9|
|          31| execute| 20240614|          5|
|          21| execute| 20240611|         10|
|          19| execute| 20240615|         15|
|          19| execute| 20240614|          5|
|          36| execute| 20240611| 

In [63]:
schema_ds_de = StructType() \
                    	.add("code", "integer")\
                    	.add("name", "string")
ds_de = spark.read.schema(schema=schema_ds_de).csv("hdfs://localhost:9010/raw_zone/fact/ds_de_vdt/danh_sach_sv_de.csv")
ds_de.show()

+----+--------------------+
|code|                name|
+----+--------------------+
|   1|          Mai Đức An|
|   2|      Nguyễn Mai Anh|
|   3|   Ngô Ngọc Tuấn Anh|
|   4|      Trần Trung Anh|
|   5|       Trần Ngọc Bảo|
|   6|  Nguyễn Vũ Hòa Bình|
|   7|    Nguyễn Thành Đạt|
|   8|        Đỗ Thành Đạt|
|   9|    Nguyễn Khoa Đoàn|
|  10|    Nguyễn Quốc Dũng|
|  11|     Đường Minh Quân|
|  12|   Dương Quang Giang|
|  13|    Nguyễn Minh Hiếu|
|  14|        Ngô Phi Hùng|
|  15|Nguyễn Đình Thiên...|
|  16|        Đỗ Doãn Khắc|
|  17|      Châu Minh Khải|
|  18|      Phạm Đình Khôi|
|  19|        Lê Bảo Khánh|
|  20|        Lê Minh Phúc|
+----+--------------------+
only showing top 20 rows



In [64]:
ans = ds_de.join(df_procesed,ds_de.code ==  df_procesed.student_code,"inner").select('timestamp','code','name','activity','total_files')
ans.show()

+---------+----+-------------------+--------+-----------+
|timestamp|code|               name|activity|total_files|
+---------+----+-------------------+--------+-----------+
| 20240613|  16|       Đỗ Doãn Khắc| execute|          9|
| 20240613|  11|    Đường Minh Quân| execute|         17|
| 20240612|  24|    Nguyễn Hoài Nam| execute|         18|
| 20240611|  12|  Dương Quang Giang| execute|         16|
| 20240613|  17|     Châu Minh Khải| execute|         13|
| 20240614|   8|       Đỗ Thành Đạt| execute|          6|
| 20240615|  31|        Tạ Đức Tiến| execute|          9|
| 20240613|  39|      Huỳnh Đoan Hồ| execute|          6|
| 20240614|  13|   Nguyễn Minh Hiếu| execute|         26|
| 20240612|  37|         Đào Anh Vũ| execute|          9|
| 20240610|  17|     Châu Minh Khải| execute|         14|
| 20240610|  32|    Đinh Việt Thành| execute|          5|
| 20240612|  19|       Lê Bảo Khánh| execute|          4|
| 20240613|   3|  Ngô Ngọc Tuấn Anh| execute|          9|
| 20240614|  3

In [81]:
names = [row.name for row in ans.select("name").distinct().collect()]
print(names)

['Huỳnh Minh Tú', 'Trần Trung Anh', 'Vũ Hữu Sỹ', 'Đỗ Doãn Khắc', 'Nguyễn Địch Nhật Minh', 'Ngô Ngọc Tuấn Anh', 'Dương Quang Giang', 'Nguyễn Mai Anh', 'Phạm Đình Khôi', 'Châu Minh Khải', 'Ngô Phi Hùng', 'Nguyễn Duy Hưng', 'Đào Thanh Nguyên', 'Lê Minh Phúc', 'Nguyễn Thành Đạt', 'Nguyễn Đình Thiên Huy', 'Đường Minh Quân', 'Nguyễn Hoài Nam', 'Tạ Đức Tiến', 'Nguyễn Thị Thùy Trang', 'Lê Bảo Khánh', 'Phùng Huy Quang', 'Nguyễn Minh Tuấn', 'Trần Phúc Mạnh Linh', 'Nguyễn Bá Thiêm', 'Nguyễn Minh Quân', 'Nguyễn Minh Hiếu', 'Lương Thị Mai Phương', 'Nguyễn Khoa Đoàn', 'Huỳnh Tấn Lộc', 'Vũ Khắc Long', 'Nguyễn Vũ Hòa Bình', 'Đinh Việt Thành', 'Huỳnh Đoan Hồ', 'Trần Ngọc Bảo', 'Mai Đức An', 'Nguyễn Quốc Dũng', 'Đỗ Thành Đạt', 'Đào Anh Vũ']


In [92]:
for name in names:
    df_filtered = ans.filter(ans.name == name)
    df_filtered.toPandas().to_csv(f"./output/{name}.csv",header=False,index=False)
    # df_filtered.write.csv()